In [1]:
from dotenv import load_dotenv
_ = load_dotenv()

In [6]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

In [7]:
tool = TavilySearchResults(max_results=4) #increased number of results
print(type(tool))
print(tool.name)

<class 'langchain_community.tools.tavily_search.tool.TavilySearchResults'>
tavily_search_results_json


In [8]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

In [9]:
class Agent:

    def __init__(self, model, tools, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges(
            "llm",
            self.exists_action,
            {True: "action", False: END}
        )
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile()
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            if not t['name'] in self.tools:      # check for bad tool name from LLM
                print("\n ....bad tool name....")
                result = "bad tool name, retry"  # instruct LLM to retry if bad
            else:
                result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [10]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""

model = ChatOpenAI(model="gpt-4o-mini")  #reduce inference cost
abot = Agent(model, [tool], system=prompt)

In [17]:
messages = [HumanMessage(content="What is the weather in new delhi?")]
result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in New Delhi'}, 'id': 'call_GmiSl3MT2ziQHDRN3DcQijv2', 'type': 'tool_call'}
Back to the model!


In [18]:
result['messages'][-1].content

'The current weather in New Delhi is as follows:\n\n- **Temperature**: Approximately 18.1°C (64.6°F)\n- **Condition**: Partly Cloudy\n- **Wind**: 3.4 mph (5.4 kph) from the North\n- **Humidity**: 49%\n- **Visibility**: 10 km\n- **Pressure**: 1017 mb\n\nThe temperature today has varied with a minimum of around 12.05°C and a maximum expected to reach 21.2°C.\n\nFor more detailed updates, you can check the sources [here](https://www.weatherapi.com/) or [Hindustan Times](https://www.hindustantimes.com/cities/delhi-news/delhi-weather-today-aqi-and-rain-forecast-updates-december-24-2024-101735003805233.html).'

In [19]:
messages = [HumanMessage(content="What is the weather in New delhi and Hyderabad?")]
result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in New Delhi'}, 'id': 'call_eHIVcSUdqe9bxwASYt7vASOf', 'type': 'tool_call'}
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Hyderabad'}, 'id': 'call_uQEwbkgJ5TEDyiW6uUQAkVVr', 'type': 'tool_call'}
Back to the model!


In [26]:
result['messages'][-1].content

The winner of the 2024 World Chess Championship is Indian Grandmaster D. Gukesh. 

The total population of Norway in 2024 is approximately 5,514,477 according to one source, while another source estimates it to be about 5,602,359 as of December 22, 2024.


In [30]:
# Note, the query was modified to produce more consistent results. 
# Results may vary per run and over time as search information and models change.

query = "Who won the world chess championship in 2024? What is the total population of the country? Answer each question." 
messages = [HumanMessage(content=query)]

model = ChatOpenAI(model="gpt-4o")  # requires more advanced model
abot = Agent(model, [tool], system=prompt)
result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': '2024 World Chess Championship winner'}, 'id': 'call_Z5mbhZrbad9s6Iu96C8N73dK', 'type': 'tool_call'}
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current population of China 2024'}, 'id': 'call_QQQJdqxyHq7ESRhT8Fs9OBKB', 'type': 'tool_call'}
Back to the model!


In [25]:
print(result['messages'][-1].content)

The winner of the 2024 World Chess Championship is Indian Grandmaster D. Gukesh. 

The total population of Norway in 2024 is approximately 5,514,477 according to one source, while another source estimates it to be about 5,602,359 as of December 22, 2024.
